In [1]:
# funcs for playing morse code
import winsound
import time


def play_morse_code(morse_code: str) -> None:
    for char in morse_code:
        match char:
            case ".":
                winsound.Beep(523, 100)
            case "-":
                winsound.Beep(523, 200)
            case " ":
                time.sleep(0.1)
            case _:
                print("invalid")


def num_to_morse(num: int) -> str:
    morse_code = []
    for num in str(num):
        match num:
            case "1":
                morse_code.append(".----")
            case "2":
                morse_code.append("..---")
            case "3":
                morse_code.append("...--")
            case "4":
                morse_code.append("....-")
            case "5":
                morse_code.append(".....")
            case "6":
                morse_code.append("-....")
            case "7":
                morse_code.append("--...")
            case "8":
                morse_code.append("---..")
            case "9":
                morse_code.append("----.")
            case "0":
                morse_code.append("-----")
    return " ".join(morse_code)

In [2]:
# Continuously update the database and check for items that can be reprocessed
import requests

from IPython.display import clear_output

REPROCESS_PARAMS = {
    "rep_yield": 0.55,
    "tax": 0.036,
    "min_roi": 0.1,
    "location_id": 60003760,
}


def print_trades(trade_list: list[dict]) -> None:
    clear_output(wait=True)
    need_to_notify = False
    max_profit = 0

    print(f"\n{'Item':<64} {'Break Even':<15} {'Buy Value':<15} {'Sell Value':<15} {'Profit':<15} {'ROI':<15}")
    for trade in trade_list:
        if not need_to_notify and trade["profit"] > 5e6:
            need_to_notify = True
            max_profit = trade["profit"]
        print(
            f"{trade['item']:<64} {trade['breakeven']:<15} {trade['buy_value']:<15} {trade['sell_value']:<15} {trade['profit']:<15} {round(trade['profit'] / trade['buy_value'] * 100, 2):<15}"
        )

    if need_to_notify:
        play_morse_code(num_to_morse(int(max_profit / 1e6)))


url = "http://127.0.0.1:8000/reprocess_trades"
r = requests.get(url, params=REPROCESS_PARAMS)
print_trades(r.json())

while True:
    time.sleep(1)
    url = "http://127.0.0.1:8000/update_tasks"
    r = requests.get(url)

    if r.status_code != 200:
        print(f"Failed to send update request: {r.status_code} {r.text}")
        time.sleep(10)
        continue

    print_reprocess = False
    for task_updated in r.json():
        if not (task_updated["task_name"] == "market.reprocess" and task_updated["successful"]):
            continue
        print_reprocess = True

    if not print_reprocess:
        continue

    url = "http://127.0.0.1:8000/reprocess_trades"
    r = requests.get(url, params=REPROCESS_PARAMS)
    print_trades(r.json())